### What is the Segmentation Image? (phân vùng ảnh)

Trong lĩnh vực thị giác máy tính (computer vision), phân vùng ảnh là một quá trình chia một bức ảnh số thành nhiều phần khác nhau (tập hợp các điểm ảnh, hay có thể gọi là superpixels). Mục tiêu của phân vùng ảnh là để đơn giản hóa và hoặc thay đổi biểu diễn của một tấm ảnh vào điều gì đó có ý nghĩa hơn và dễ dàng để phân tích.

Một vài ứng dụng cụ thể trong phân vùng ảnh:

+ Hình ảnh y tế
+ Nhận dạng đối tượng
    + Phát hiện đi bộ
    + Phát hiện khuôn mặt
    + Phát hiện đèn dừng xe
+ Xác định vị trí đối tượng trong ảnh vệ tinh
+ Một số nhiệm vụ nhận dạng
    + Nhận dạng khuôn mặt
    + Nhận dạng vân tay
    + Nhận dạng mắt
+ Hệ thống giám sát giao thông
+ Camera giám sát an ninh

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/03/instance_segmentation_example.jpg)

Nguồn: cs231n.stanford.edu

### How to work?

Khác với bài toán object detection là tìm một box bao quanh một đối tượng. Nên vấn đề là box có thể có phần dư, và một khái niệm mới ra đời để khắc phục vấn đề cho bài toán object detection là segmentation image. 

Bằng việc xem xét một hình ảnh gồm có nhiều đối tượng, một đối tượng là một khu vực trong ảnh, bằng cách cố gắng xác định xem 1 pixel trong ảnh đang thuộc vùng của đối tượng nào. Khi đó bài toán thuộc bài toán classification điểm ảnh vào n vùng.

### Tại sao cần Segmentation Image?

Trong mô hình đang cố phân loại các dựa trên cấp độ pixel vì vậy mà khả năng chính xác khi phân khúc đem đến lợi thế so với các mô hình trong bài toán object detection dừng ở cấp độ bouding box. 

Ví dụ:

Ung thư từ lâu đã là một căn bệnh chết người. Ngay cả trong thời đại tiến bộ công nghệ ngày nay, ung thư có thể gây tử vong nếu chúng ta không xác định nó ở giai đoạn đầu. Phát hiện (các) tế bào ung thư càng nhanh càng tốt có khả năng cứu sống hàng triệu người.

Hình dạng của các tế bào ung thư đóng một vai trò quan trọng trong việc xác định mức độ nghiêm trọng của ung thư. Bạn có thể đã đặt các mảnh lại với nhau - phát hiện đối tượng sẽ không hữu ích ở đây. Chúng tôi sẽ chỉ tạo các hộp giới hạn sẽ không giúp chúng tôi xác định hình dạng của các ô.

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/03/cancer-cell-segmentation.png)




Các thuật toán tốt cho Segmentation Image:

* [1]: Rethinking Atrous Convolution for Semantic Image Segmentation #3 best model for Semantic Segmentation on PASCAL VOC 2012 test

* [2]: U-net

* [3]: SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation

### KHám phá U-Net

U-net được phát triển bởi Olaf Ronneberger et al. cho phân đoạn hình ảnh y tế sinh học. Kiến trúc chứa hai đường dẫn. Đường dẫn đầu tiên là đường dẫn co lại (còn được gọi là bộ mã hóa) được sử dụng để chụp bối cảnh trong ảnh. Bộ mã hóa chỉ là một chồng truyền thống của các lớp gộp và tối đa. Đường dẫn thứ hai là đường dẫn mở rộng đối xứng (còn được gọi là bộ giải mã) được sử dụng để cho phép nội địa hóa chính xác bằng cách sử dụng các cấu trúc chuyển vị. Do đó, nó là một mạng chập hoàn toàn từ đầu đến cuối (FCN), tức là nó chỉ chứa các lớp Convolutional và không chứa bất kỳ lớp Dense nào vì nó có thể chấp nhận hình ảnh có kích thước bất kỳ.


![](https://miro.medium.com/max/1280/1*OkUrpDD6I0FpugA_bbYBJQ.png)

![](https://miro.medium.com/max/1632/1*yzbjioOqZDYbO6yHMVpXVQ.jpeg)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model